<a href="https://colab.research.google.com/github/royerlozano/CUADERNO-DE-ROYER/blob/main/Guia_2_Taxonomia_de_Flynn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El siguiente código va a permitir que todo código ejecutado en el colab pueda ser medido

In [19]:
!pip install ipython-autotime

%load_ext autotime

time: 166 µs


In [ ]:
print(sum(range(10)))

45
time: 1.01 ms


Pregunta #1: Que porción de 1 segundo es el valor impreso?

Es milisegundo


---



A seguir, tenemos una librería de Python llamado **numba** que realiza paralelización automatica. Asi, se puede verificar que al usar prange() se tiene mejor tiempo de ejecución que al usar range()

In [20]:
#CAmbiando datos 
#para A=np.arange(4,4) = 9
#time: 401 ms
#se ejecuta en mas tiempo(paralelamente)
from numba import njit, prange
import numpy as np

A = np.arange(2, 5)
@njit(parallel=True)
def prange_test(A):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in prange(A.shape[0]):
        s += A[i]
    return s

print(prange_test(A))

9
time: 401 ms


In [21]:
#Cambiamos los datos 
#para A=np.arange(4,4) = 7
#time: 7.09 ms
#el tiempo de ejecucion es mucho menor
from numba import njit, prange
import numpy as np

A = np.arange(3, 5)
#@njit(parallel=True)
def prange_test(A):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in range(A.shape[0]):
        s += A[i]
    return s

print(prange_test(A))



7
time: 7.09 ms


Pregunta #2: identifique otros valores en A, de manera que, serializando, tengamos mejor resultado que paralelizando



---



La Taxonomia de Flynn define 4 tipos de arquitecturas para computación paralela: SISD, SIMD, MISD, y MIMD. 


---


Pregunta #3 : El ultimo código ejecutado es de tipo?



---



Pregunta #4: ¿Segun la Tax. de Flynn, De que tipo es el siguiente código paralelo? Comentar el código para justificar su respuesta

In [ ]:
#SIMD(simple instruccion multipe data)
#tenemos dos hilos que vienen a ser los hilos T1 y T2 y tenemos como datos sus nombres 
#Thread-1 y Thread-2 y los numeros de los hilos 1 y 2
#una sola instruccion que vendria a ser la funcion print_time
import threading
import time

def print_time(name, n):
    count = 0   
    print("Para el Hilo: %s, en el momento: %s, su valor de count es: %s" % ( name, time.ctime(), count))
    while count < 5:
        time.sleep(n)
        count+=1
        print("%s: %s. count %s" % ( name, time.ctime(), count))

        
t1 = threading.Thread(target=print_time, args=("Thread-1", 0, ) )
t2 = threading.Thread(target=print_time, args=("Thread-2", 0, ) )

t1.start()
t2.start()

Para el Hilo: Thread-1, en el momento: Wed Dec  9 15:26:34 2020, su valor de count es: 0Para el Hilo: Thread-2, en el momento: Wed Dec  9 15:26:34 2020, su valor de count es: 0
Thread-1: Wed Dec  9 15:26:34 2020. count 1
Thread-1: Wed Dec  9 15:26:34 2020. count 2
Thread-1: Wed Dec  9 15:26:34 2020. count 3
Thread-1: Wed Dec  9 15:26:34 2020. count 4
Thread-1: Wed Dec  9 15:26:34 2020. count 5
time: 6.49 ms

Thread-2: Wed Dec  9 15:26:34 2020. count 1
Thread-2: Wed Dec  9 15:26:34 2020. count 2
Thread-2: Wed Dec  9 15:26:34 2020. count 3
Thread-2: Wed Dec  9 15:26:34 2020. count 4
Thread-2: Wed Dec  9 15:26:34 2020. count 5




---



Una computadora paralela tipo MIMD es utilizado más en la computación distribuida, ejm. Clusters. El siguiente código en python desktop muestra tal funcionamiento

In [ ]:
#greeting-server.py
import Pyro4

@Pyro4.expose
class GreetingMaker(object):
    def get_fortune(self, name):
        return "Hello, {0}. Here is your fortune message:\n" \
               "Behold the warranty -- the bold print giveth and the fine print taketh away.".format(name)

daemon = Pyro4.Daemon()                # make a Pyro daemon
uri = daemon.register(GreetingMaker)   # register the greeting maker as a Pyro object

print("Ready. Object uri =", uri)      # print the uri so we can use it in the client later
daemon.requestLoop()                   # start the event loop of the server to wait for calls

In [ ]:
#greeting-client.py
import Pyro4

uri = input("What is the Pyro uri of the greeting object? ").strip()
name = input("What is your name? ").strip()

greeting_maker = Pyro4.Proxy(uri)         # get a Pyro proxy to the greeting object
print(greeting_maker.get_fortune(name))   # call method normally

Pregunta #5: Explique que hace este código de tipo MIMD
el modulo del servidores es quien acepta las multiples instrucciones ejecutados en un segundo plano y el modulo del cliente es quien acepta los multiples datos; es decir; se crea multiples instancias del modulo del cliente para enviar multiples datos.
---





---



Ejercicio Propuesto: Crear un ejemplo que muestre una computación paralela de tipo MISD

In [27]:
#MISD (multiple instruccion simple data)
#tenemos un hilo que vienen a ser los hilos T1 y tenemos como dato su nombre 
#Thread-1 y el numero del hilo 1.
#haremos uso de la funcion print_time y Mostrar 2 instrucciones.
import threading
import time
def Mostrar(name,contador,n):
    time.sleep(n)
    print("%s: %s. count %s" % ( name, time.ctime(), contador))

def print_time(name,n):
  contador=0
  print("Para el Hilo: %s, en el momento: %s, el valor del contador es: %s" % ( name, time.ctime(), contador))
  while contador <= 3:
        contador+=1
        Mostrar(name,contador,n)
t1 = threading.Thread(target=print_time, args=("Thread-1", 2, ) )
t1.start()






Para el Hilo: Thread-1, en el momento: Thu Dec 10 05:49:44 2020, el valor del contador es: 0
time: 7.5 ms


**Referencias**

https://wiki.python.org/moin/ParallelProcessing

https://numba.readthedocs.io/en/stable/user/parallel.html

https://ao.gl/how-to-measure-execution-time-in-google-colab/

http://noisymime.org/blogimages/SIMD.pdf